In [5]:
import os, pickle
from operator import itemgetter
import numpy as np
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
entity_ranking_op_folder = '../../../Data/output/Entity_Ranking'
type_rep_folder = 'TypeRep/type_word'
type_vec_folder = 'TypeVectors'
entity_rep_folder = 'EntityRep'
entity_context_word_file = 'doc_entity_context_word.p'
per_type_rep_file_name = 'per_tag_vec_dict.p'
org_type_rep_file_name = 'org_tag_vec_dict.p'
loc_type_rep_file_name = 'loc_tag_vec_dict.p'

word_emnbedding_pretrained_trained_on_corpus = '../../../word_embeddings/trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt'

entity_context_word_file = os.path.join(entity_ranking_op_folder, entity_rep_folder, entity_context_word_file)
per_type_rep_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_vec_folder, per_type_rep_file_name)
org_type_rep_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_vec_folder, org_type_rep_file_name)
loc_type_rep_file = os.path.join(entity_ranking_op_folder, type_rep_folder, type_vec_folder, loc_type_rep_file_name)

In [3]:
tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others', 'ORG_Accused', 'ORG_Victim', 'ORG_Others', 'PER_Victim', 'PER_Others', 'PER_Accused']
per_tag_list = ['PER_Victim', 'PER_Others', 'PER_Accused']
loc_tag_list = ['LOC_Event', 'LOC_Accused', 'LOC_Victim', 'LOC_Others']
org_tag_list = ['ORG_Accused', 'ORG_Victim', 'ORG_Others']

In [6]:
entity_rep_context_word_file_dict = pickle.load(open(entity_context_word_file, 'rb'))
per_type_rep_dict = pickle.load(open(per_type_rep_file, 'rb'))
org_type_rep_dict = pickle.load(open(org_type_rep_file, 'rb'))
loc_type_rep_dict = pickle.load(open(loc_type_rep_file, 'rb'))

word_vectors = KeyedVectors.load_word2vec_format(word_emnbedding_pretrained_trained_on_corpus, binary=False)

In [7]:
word_vector = word_vectors.get_vector('Narendra')
size = len(word_vector)

In [17]:
def group_average(entity, role):
    num_of_words_in_entity = len(entity)
    num_of_words_in_role = 1
    total_number_of_words = num_of_words_in_entity + num_of_words_in_role
    
    sum_of_entity_type_word_vec = np.zeros(size)
    for word in entity:
        word_vec = word_vectors.get_vector(word)
        word_vec = word_vec/np.linalg.norm(word_vec)
        sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, word_vec)
    role = role/np.linalg.norm(role)
    sum_of_entity_type_word_vec = np.add(sum_of_entity_type_word_vec, role)
    
    dot_product = np.dot(sum_of_entity_type_word_vec, sum_of_entity_type_word_vec)
    group_avg = float(dot_product - (total_number_of_words))/(total_number_of_words * (total_number_of_words - 1))
    return group_avg    
    

In [20]:
precision1 = dict()
doc_count_for_tag = dict()

for tag in tag_list:
    doc_count_for_tag[tag] = 0
    precision1[tag] = 0
    

# Precision at K
K = 2
count = 0
for doc_id in entity_rep_context_word_file_dict:
    #print(doc_id)
    num_actual_entities_with_role = dict()
    if count > 0:
        break
    per_entities = list()
    loc_entities = list()
    org_entities = list()
    doc_role_dict = entity_rep_context_word_file_dict[doc_id]
    for role in doc_role_dict.keys():
        entities = doc_role_dict[role]
        num_actual_entities_with_role[role] = len(entities)
        if role in per_tag_list:
            for entity in entities:
                per_entities.append((entity, role))
        if role in org_tag_list:
            for entity in entities:
                org_entities.append((entity, role))
        if role in loc_tag_list:
            for entity in entities:
                loc_entities.append((entity, role))
    
    num_per_entity = len(per_entities)
    num_org_entity = len(org_entities)
    num_loc_entity = len(loc_entities)
    
    for role in per_tag_list:
        role_vector = per_type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in per_entities:
                sim = 2
                sim = group_average(entity[0], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_per_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_per_entity)
                
    for role in loc_tag_list:
        role_vector = loc_type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in loc_entities:
                sim = 2
                sim = group_average(entity[0], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_loc_entity)):
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_loc_entity)
                
    for role in org_tag_list:
        role_vector = org_type_rep_dict[role]
        retrivedResult = list()
        if num_actual_entities_with_role[role] != 0:
            TP = 0
            doc_count_for_tag[role] = doc_count_for_tag[role] + 1
            for entity in org_entities:
                sim = 2
                sim = group_average(entity[0], role_vector)
                retrivedResult.append((entity, sim))
            retrivedResult = sorted(retrivedResult,key=itemgetter(1), reverse=True)
            for i in range(min(K, num_org_entity)):
                
                #print(len(retrivedResult))
                #print(role)
                #print(num_actual_entities_with_role)
                if retrivedResult[i][0][1] == role:
                    TP = TP + 1
            if TP ==  num_actual_entities_with_role[role]:
                precision1[role] = precision1[role] + 1
            else:
                precision1[role] = precision1[role] + float(TP)/min(K, num_org_entity)
    #print(num_actual_entities_with_role)
    #count = 1

for tag in tag_list:
    if doc_count_for_tag[tag] > 0:
        print(tag, float(precision1[tag] * 100)/doc_count_for_tag[tag], '%')

LOC_Event 48.63088718510405 %
LOC_Accused 35.273972602739725 %
LOC_Victim 50.0 %
LOC_Others 66.34920634920636 %
ORG_Accused 55.3125 %
ORG_Victim 50.0 %
ORG_Others 81.11964873765093 %
PER_Victim 54.26829268292683 %
PER_Others 80.57210965435041 %
PER_Accused 67.93103448275862 %


In [41]:
from operator import itemgetter
data = [('abc', 1.21),('abc', 2.31),('abc', 1.48), ('abc',2.21)]
data = sorted(data,key=itemgetter(1), reverse=True)
data[0:4]

[('abc', 2.31), ('abc', 2.21), ('abc', 1.48), ('abc', 1.21)]

In [35]:
a = np.array([1,2,3])
b = np.array([1,2,3])
cosine_distance(a,b)


0.0